# Start-RscManagedVolumeSnapshot
Managed Volumes (MV) provide hosts with a backup target location on a Rubrik cluster. The cluster manages snapshots of the data through the policies of a specified SLA Domain. DBAs will typically use a MVs for writing backups of databases where Rubrik does not have a built in snappable yet. The Start-RscManagedVolumeSnapshot cmdlet enables the MV for write allowing a DBA to write directly to the share on the Rubrik storage from their host. 

This process works with Persistent Mount Managed Volumes and not SLA Driven Managed Volumes. 

# Workflow
To get information about the Managed Volume, you need to do the following
1. Use **Connect-RSC** to Connect to RSC
1. Use **Get-RscCluster** to get the cluster id that the Managed Volume is registered. This is akin to the datacenter that the Managed Volume is located in. While you can run the next command without the cluster, you may get more records back than expected. This is because your Managed Volume could exist on 2 different clusters. Once as a replicated source and once as the replicated target. 
1. Use **Get-RscManagedVolume** to get the id of the Managed Volume
1. Use **Start-RscManagedVolumeSnapshot** to make the MV writeable
1. Use **Disconnect-RSC** to disconnect from RSC

## Connect to RSC

In [1]:
Import-Module RubrikSecurityCloud
Connect-RSC


ClientId                                AuthenticationState SdkSchemaVersion ServerVersion
--------                                ------------------- ---------------- -------------
client|kB5og68XXrFlfXcvWhn3jlHdf1OKyJCB          AUTHORIZED v20240219-20     v20240219-20



## Get the Cluster ID

In [2]:
$ClusterName = "vault-r-london"
$RscCluster = Get-RscCluster -Name $ClusterName

## Get the Managed Volume Information

In [3]:
$MVName = "rp-mysql-01"
$RscManagedVolume = Get-RscManagedVolume -Name $MVName -RscCluster $RscCluster

## Make the Mananged Volume writeable

In [4]:
Start-RscManagedVolumeSnapshot -RscManagedVolume $RscManagedVolume


OwnerId SnapshotId                           AsyncRequestStatus
------- ----------                           ------------------
        f69fd30e-3b48-4d54-bb9d-d08c2bb989dd RubrikSecurityCloud.Types.AsyncRequestStatus



## Send data to open MV
The above command opened the MV for write. Now, you can use any command you want to send data to the MV. 
- Oracle DBAs may use RMAN scripts
- SQL Server DBAs may use T-SQL scripts
- MySQL DBAs may use mysqldump
- PostgreSQL DBAs may use barman

---

# Disconnect from RSC
It is important to have each script you create end by disconnecting from RSC. If you do not, then you will run into issues with token authentication exhaustion. RSC allows a Service Account to have 10 concurrent open connections to RSC. Additional requests will be denied until existing connections are closed. 

You can easily disconnect your connection by running the below statement. 

In [5]:
Disconnect-Rsc

The Rubrik Security Cloud session has been terminated.
